# Some numbers

In [31]:
import pandas as pd
from pathlib import Path
import glob
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import resolve1

## Count Protokolle and Pages

In [32]:
docs = 0
pages = 0

for f in glob.glob(str(Path('./../export/Files/*.pdf'))):

    parser = PDFParser(open(f, 'rb'))
    document = PDFDocument(parser)
    docs += 1
    pages += resolve1(document.catalog['Pages'])['Count']

print("Dokumente: %s" % docs)
print("Seiten: %s" % pages)

Dokumente: 1135
Seiten: 78433


In [33]:
df = pd.concat([
    pd.read_csv(Path('../export/votum/votum_0.csv')),
    pd.read_csv(Path('../export/votum/votum_1.csv'))
])

print("Voten: %s" % len(df))
print("First: %s" % df['sitzung_date'].min())
print("Last: %s" % df['sitzung_date'].max())

Voten: 80306
First: 2001-01-08
Last: 2022-03-28


## Count Geschäfte

In [30]:
df = pd.read_csv(Path('../export/geschaefte.csv'), encoding='UTF-8')


print("Geschäfte: %s" % len(df))
print("First: %s" % df['start'].min())
print("Last: %s" % df['start'].max())

Geschäfte: 10614
First: 2001-01-01
Last: 2022-04-14


In [29]:
df.head()

,krnr,vorlagenr,titel,geschaeftsart,behandelndekommission,behandelndekommissionkurzname,direktion,direktionKurzname,start,end,...,status,erstunterzeichnervorname,erstunterzeichnername,erstunterzeichneristkantonsrat,erstunterzeichnerpartei,letzterschrittstart,letzterschritttyp,letzterschritttext,geschlecht,jahrgang
0,334/2001,NaN,Fakultatives Referendum (Flughafenfondsgesetz;...,Diverses,NaN,NaN,NaN,NaN,2001-01-01,2001-01-02,...,Erledigt,NaN,"Kantonsrat Zürich, Kommission",False,NaN,2001-11-19,Zustimmung,Zustimmung,NaN,NaN
1,34/2001,NaN,Erlass eines Taxigesetzes,Einzelinitiative,NaN,NaN,NaN,NaN,2001-01-01,2001-01-02,...,Erledigt,Peter,Vögeli,False,NaN,2001-06-11,Ablehnung - Nicht vorläufig unterstützt,Nicht vorläufig unterstützt (11 Stimmen),NaN,NaN
2,339/2001,NaN,Fachhochschule für die Berufe des Gesundheitsw...,Anfrage,NaN,NaN,NaN,NaN,2001-01-01,2001-01-02,...,Erledigt,Erika,Ziltener,True,SP,2002-01-23,Antwort Regierungsrat,Antwort RR,w,1955.0
3,338/2001,NaN,Umsetzung des Punktspielautomatenverbots,Anfrage,NaN,NaN,NaN,NaN,2001-01-01,2001-01-02,...,Erledigt,Hanspeter,Amstutz,True,EVP,2002-01-23,Antwort Regierungsrat,Antwort RR,m,1947.0
4,337/2001,NaN,Veranwortlichkeitsklage gegen die zuständigen ...,Dringliche Anfrage,NaN,NaN,NaN,NaN,2001-01-01,2001-01-02,...,Erledigt,Anna Maria,Riedi,True,SP,2001-11-21,Antwort Regierungsrat,Antwort RR,w,1962.0
